# Stock NeurIPS2018 Part 2. Train
This series is a reproduction of *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the second part of the NeurIPS2018 series, introducing how to use FinRL to make data into the gym form environment, and train DRL agents on it.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\jason\appdata\local\temp\pip-req-build-eel5o1w6
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 9e8c38aa5b92bbf0e20f65fc611fd43b43196859
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\jason\appdata\local\temp\pip-install-r0ekff0k\elegantrl_1f58174a581b4034a6a9b83445b87222
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 37aac1f592e1add9f9fd37ae8db1094656009b76
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\Jason\AppData\Local\Temp\pip-req-build-eel5o1w6'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\Jason\AppData\Local\Temp\pip-install-r0ekff0k\elegantrl_1f58174a581b4034a6a9b83445b87222'


In [2]:
import numpy as np
import pandas as pd
from stable_baselines3.common.logger import configure
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories

import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parents[1]))
from finai_contest.env_stock_trading.env_stock_trading_meta import StockTradingEnv_FinRLMeta
from finai_contest.env_stock_trading.env_stock_trading_gym_anytrading import StockTradingEnv_gym_anytrading
check_and_make_directories([TRAINED_MODEL_DIR])

# Part 2. Build A Market Environment in OpenAI Gym-style

rl_diagram_transparent_bg.png

The core element in reinforcement learning are **agent** and **environment**. You can understand RL as the following process: 

The agent is active in a world, which is the environment. It observe its current condition as a **state**, and is allowed to do certain **actions**. After the agent execute an action, it will arrive at a new state. At the same time, the environment will have feedback to the agent called **reward**, a numerical signal that tells how good or bad the new state is. As the figure above, agent and environment will keep doing this interaction.

The goal of agent is to get as much cumulative reward as possible. Reinforcement learning is the method that agent learns to improve its behavior and achieve that goal.

To achieve this in Python, we follow the OpenAI gym style to build the stock data into environment.

state-action-reward are specified as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes the price data and technical indicators based on the past data. It will learn by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 constituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.

## Read data

We first read the .csv file of our training data into dataframe.

In [3]:
train = pd.read_csv('./data/train_data.csv')
# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
train = train[train["tic"] == "AAPL"]
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2009-01-02,AAPL,2.724325,2.733032,2.556513,2.578127,7.460152e+08,4.0,0.000000,2.944416,2.619211,100.000000,66.666667,100.000000,2.724325,2.724325,39.189999,0.000000
1,2009-01-05,AAPL,2.839302,2.887335,2.783164,2.796974,1.181608e+09,0.0,0.002580,2.944416,2.619211,100.000000,66.666667,100.000000,2.781814,2.781814,39.080002,0.000000
2,2009-01-06,AAPL,2.792471,2.917054,2.773558,2.880430,1.289310e+09,1.0,0.001835,2.901000,2.669732,70.355312,45.847437,100.000000,2.785366,2.785366,38.560001,0.000000
3,2009-01-07,AAPL,2.732131,2.776861,2.709616,2.756147,7.530488e+08,2.0,-0.000728,2.880446,2.663668,50.429352,-30.767106,43.608273,2.772057,2.772057,43.389999,0.000000
4,2009-01-08,AAPL,2.782865,2.796374,2.703011,2.714719,6.735008e+08,3.0,-0.000086,2.868583,2.679855,60.227087,-8.239411,48.358311,2.774219,2.774219,42.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3268,2021-12-27,AAPL,176.796036,176.884268,173.599928,173.619525,7.491960e+07,0.0,4.966004,179.988873,156.216783,65.037365,111.207256,48.878374,163.182837,153.508200,17.680000,9.832529
3269,2021-12-28,AAPL,175.776428,177.776459,175.031327,176.629389,7.914430e+07,1.0,5.015728,180.195296,157.878025,64.013434,108.220225,50.226012,164.140365,154.110296,17.540001,8.468392
3270,2021-12-29,AAPL,175.864670,177.090173,174.648965,175.815647,6.234890e+07,2.0,5.004565,180.775805,158.677922,64.064086,99.806902,48.433418,165.100506,154.771132,16.950001,6.345802


## Construct the environment

Calculate and specify the parameters we need for constructing the environment.

In [4]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [6]:
env_used = "finrl"
# env_used = "gym_anytrading"
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension


if env_used =="finrl":
    env_kwargs = {
        "hmax": 100,
        "initial_amount": 1000000,
        "num_stock_shares": num_stock_shares,
        "buy_cost_pct": buy_cost_list,
        "sell_cost_pct": sell_cost_list,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": INDICATORS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4
    }


    e_train_gym = StockTradingEnv_FinRLMeta(df = train, **env_kwargs)

elif env_used =="gym_anytrading":
    env_kwargs = {
    "hmax": np.inf,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": 2*stock_dimension,
    "reward_scaling": 1e-4,
    "window_size": 30
    }

    e_train_gym = StockTradingEnv_gym_anytrading(df = train, **env_kwargs)

## Environment for training

In [7]:
env_train, _ = e_train_gym.get_sb_env()

In [8]:
# train_aapl = train[train["tic"] == "aapl"]
# train_gym_anytrade = pd.DataFrame()
# train_gym_anytrade['Time'] = pd.to_datetime(train_aapl['date'])  # Convert to datetime
# train_gym_anytrade['Open'] = train_aapl['open']
# train_gym_anytrade['High'] = train_aapl['high']
# train_gym_anytrade['Low'] = train_aapl['low']
# train_gym_anytrade['Close'] = train_aapl['close']
# train_gym_anytrade['Volume'] = train_aapl['volume']
# train_aapl

In [9]:
# import gymnasium as gym
# import gym_anytrading
# env_train = gym.make(
#     'stocks-v0',
#     df=train_gym_anytrade,
#     window_size=30,
#     frame_bound=(30, len(train_gym_anytrade))
# )
# from stable_baselines3.common.vec_env import DummyVecEnv

# def get_sb_env(self):
#     e = DummyVecEnv([lambda: self])
#     obs = e.reset()
#     return e, obs
# # Patch the method
# env_train = env_train.env
# env_train = env_train.env
# env_train.get_sb_env = get_sb_env.__get__(env_train)
# env_train

# Part 3: Train DRL Agents
* Here, the DRL algorithms are from **[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)**. It's a library that implemented popular DRL algorithms using pytorch, succeeding to its old version: Stable Baselines.
* Users are also encouraged to try **[ElegantRL](https://github.com/AI4Finance-Foundation/ElegantRL)** and **[Ray RLlib](https://github.com/ray-project/ray)**.

In [10]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = False
if_using_ddpg = False
if_using_ppo = True
if_using_td3 = False
if_using_sac = False

## Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [11]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [12]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

In [13]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None

### Agent 2: DDPG

In [14]:
# agent = DRLAgent(env = env_train)
# model_ddpg = agent.get_model("ddpg")

# if if_using_ddpg:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/ddpg'
#   new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_ddpg.set_logger(new_logger_ddpg)

In [15]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

In [16]:
trained_ddpg.save(TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None

### Agent 3: PPO

In [17]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to results/ppo


In [19]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000) if if_using_ppo else None

--------------------------------------
| time/              |               |
|    fps             | 1086          |
|    iterations      | 1             |
|    time_elapsed    | 1             |
|    total_timesteps | 2048          |
| train/             |               |
|    learning_rate   | 0.00025       |
|    reward          | 0.0           |
|    reward_max      | 0.060199596   |
|    reward_mean     | 0.00015746405 |
|    reward_min      | -0.06868129   |
--------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 556           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.0053390255  |
|    clip_fraction        | 0.0386        |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.42         |
|    expl

In [20]:
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo_"+env_used) if if_using_ppo else None

### Agent 4: TD3

In [24]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

# if if_using_td3:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/td3'
#   new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_td3.set_logger(new_logger_td3)

In [21]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

In [22]:
trained_td3.save(TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None

### Agent 5: SAC

In [27]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 100000,
#     "learning_rate": 0.0001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

# if if_using_sac:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/sac'
#   new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_sac.set_logger(new_logger_sac)

In [23]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=70000) if if_using_sac else None

In [24]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

## Save the trained agent
Trained agents should have already been saved in the "trained_models" drectory after you run the code blocks above.

For Colab users, the zip files should be at "./trained_models" or "/content/trained_models".

For users running on your local environment, the zip files should be at "./trained_models".